# Assignment 1 - Part of Speech Tagging

## Dependencies

In [689]:
# !pip install lightning
# !pip install torchtext.data
# !pip install torchtext
# !pip install torch

In [690]:
# TODO: remove unused dependencies

# file management
import sys
import shutil
import urllib
import tarfile
from pathlib import Path
import zipfile
import os

# DL framework
from torchmetrics import Accuracy, F1Score
from lightning.pytorch.loggers import TensorBoardLogger

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

#### Download the corpus

In [691]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)


def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")


def extract_dataset(download_path: Path, extract_path: Path):
    print("Extracting dataset... (it may take a while...)")
    with zipfile.ZipFile(download_path, 'r') as zip_file:
        zip_file.extractall(extract_path)

    print("Extraction completed!")

    Path.unlink(download_path)
    print("Deleted .zip dataset file")

In [692]:
url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

print(f"Current work directory: {Path.cwd()}")

dataset_folder = Path.cwd().joinpath("Datasets")

if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_zip_path.exists():
    download_dataset(dataset_zip_path, url)

if not dataset_path.exists():
    extract_dataset(dataset_zip_path, dataset_folder)


Current work directory: C:\Users\Matteo\Documents\University\NLP\A1


#### Encode the corpus into a pandas.DataFrame object and split it into train, validation and test sets

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

In [693]:
dataframe_rows = []  # list for DataFrame rows
id = 0

for i, file_path in enumerate(sorted(dataset_path.iterdir())):
    if file_path.is_file():  # split corpus documents in the tree categories: train, validation, tests
        if 1 <= i + 1 <= 100:
            split = 'train'
        elif 101 <= i + 1 <= 150:
            split = 'validation'
        else:
            split = 'test'

        with file_path.open(mode='r', encoding='utf-8') as text_file:  # read corpus lines
            lines = text_file.readlines()

        for line in lines:
            fields = line.strip().split('\t')
            if len(fields) == 1:
                id = id + 1
            if len(fields) >= 2:
                text = fields[0]  # store the first field as 'text'
                POS = fields[1]  # store the second field as 'POS'
                dataframe_row = {  #build DataFrame rows
                    "text": text,
                    "POS": POS,
                    "split": split,
                    "id": id
                }

                dataframe_rows.append(dataframe_row)  #append rows
# corpus DataFrame
corpus_df = pd.DataFrame(dataframe_rows)

## TASK 1: Corpus

* **Download** the corpus.
* **Encode** the corpus into a pandas.DataFrame object.
* **Split** it in training, validation, and test sets.

#### Data inspection

In [694]:
corpus_df.head(10)

,text,POS,split,id
0,Pierre,NNP,train,0
1,Vinken,NNP,train,0
2,",",",",train,0
3,61,CD,train,0
4,years,NNS,train,0
5,old,JJ,train,0
6,",",",",train,0
7,will,MD,train,0
8,join,VB,train,0
9,the,DT,train,0


In [695]:
# Train, test, validation split
df_train = corpus_df[corpus_df['split'] == 'train'].drop(columns=['split'])
df_test = corpus_df[corpus_df['split'] == 'test'].drop(columns=['split'])
df_val = corpus_df[corpus_df['split'] == 'validation'].drop(columns=['split'])

In [696]:
print("Dataframe structure:")
print(corpus_df)
print()

print("Total rows %d" % (len(corpus_df)))
print()

Dataframe structure:
          text  POS  split    id
0       Pierre  NNP  train     0
1       Vinken  NNP  train     0
2            ,    ,  train     0
3           61   CD  train     0
4        years  NNS  train     0
...        ...  ...    ...   ...
94079  quarter   NN   test  3715
94080       of   IN   test  3715
94081     next   JJ   test  3715
94082     year   NN   test  3715
94083        .    .   test  3715

[94084 rows x 4 columns]

Total rows 94084


## TASK 2: Text encoding

* Embed words using **GloVe embeddings**.
* TODO: see if we want to do it, otherwise remove it -> [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

### Embed words unsing GloVe embeddings

In [697]:
from torchtext.vocab import GloVe, build_vocab_from_iterator

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from lightning.pytorch import loggers as pl_logger

In [698]:
def load_embedding_model(embedding_dimension: int = 300):
    emb_model = GloVe(name="6B", dim=embedding_dimension)
    return emb_model

In [699]:
iterator = ([text] for text in corpus_df["POS"].unique())
vocab = build_vocab_from_iterator(iterator)
vocab.append_token("<PAD>")

pos_padding_value = vocab["<PAD>"]


class CorpusDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, embedder):
        min_id = dataframe['id'].min()
        dataframe['id'] = dataframe['id'] - min_id
        self.dataframe = dataframe.groupby("id")
        self.embedder = embedder

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sentence = self.dataframe.get_group(idx)
        text = sentence['text'].to_list()
        POS = sentence['POS'].to_list()

        POS = torch.Tensor([vocab[token] for token in POS])

        # POS_one_hot = torch.nn.functional.one_hot(POS.to(torch.int64), num_classes=len(vocab))
        # TODO: remove
        embedded_text = self.embedder.get_vecs_by_tokens(text)

        return embedded_text, POS


In [700]:
# Definition of the dataset
EMBEDDING_DIM = 50
embedder = load_embedding_model(EMBEDDING_DIM)
dataset_train = CorpusDataset(df_train, embedder)
dataset_test = CorpusDataset(df_test, embedder)
dataset_val = CorpusDataset(df_val, embedder)
dataset_all = CorpusDataset(corpus_df, embedder)


# TODO - test if it works in the LSTM training
def my_collate(batch):
    sequences, labels = zip(*batch)

    # max_len = max([len(seq) for seq in sequences])
    # sequences_padded = [seq + ["<PAD>"] * (max_len - len(seq)) for seq in sequences]

    sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pos_padding_value)

    sequences_padded = sequences_padded.type(torch.float)
    labels_padded = labels_padded.type(torch.long)

    return [sequences_padded, labels_padded]


train_loader = DataLoader(dataset_train, batch_size=32, collate_fn=my_collate)
val_loader = DataLoader(dataset_val, batch_size=32, collate_fn=my_collate)
test_loader = DataLoader(dataset_test, batch_size=32, collate_fn=my_collate)

## TASK 3: Model definition

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

### Baseline model: Bidirectional LSTM + Dense layer

In [701]:
from torchmetrics import Metric
from torchmetrics import ConfusionMatrix


class F1ScoreCustom(Metric):
    def __init__(self, num_classes: int, pos_padding_value: int = pos_padding_value):
        super().__init__()

        self.num_classes = num_classes
        self.pos_padding_value = pos_padding_value
        self.add_state("true_positive", default=torch.zeros([num_classes]), dist_reduce_fx="sum")
        self.add_state("false_negative", default=torch.zeros([num_classes]), dist_reduce_fx="sum")
        self.add_state("false_positive", default=torch.zeros([num_classes]), dist_reduce_fx="sum")

    def update(self, y_hat_class: torch.Tensor, y_class: torch.Tensor):
        confusion_matrix_metric = ConfusionMatrix(num_classes=self.num_classes, task="multiclass")
        confusion_matrix = confusion_matrix_metric(y_hat_class, y_class)

        # # Confusion matrix, TP, FN and FP for class 0 
        # #   TRUE LABEL
        # #   0               TP     FN    FN     FN      FN        
        # #   1               FP       
        # #   2               FP               
        # #   3               FP                       
        # #   4               FP                                
        # # PREDICTED LABEL   0       1      2       3       4
        # 

        true_positive = torch.Tensor([confusion_matrix[i][i] for i in range(self.num_classes)])
        false_negative = torch.Tensor([sum(confusion_matrix[i, :]) - true_positive[i] for i in range(self.num_classes)])
        false_positive = torch.Tensor([sum(confusion_matrix[:, i]) - true_positive[i] for i in range(self.num_classes)])

        self.true_positive += true_positive
        self.false_negative += false_negative
        self.false_positive += false_positive

    def compute(self):
        precision = self.true_positive / (self.true_positive + self.false_positive)
        recall = self.true_positive / (self.true_positive + self.false_negative)

        f1 = 2 * (precision * recall) / (precision + recall)

        mask = torch.ones_like(f1)
        mask[self.pos_padding_value] = 0

        f1 = f1 * mask

        return f1



In [702]:
class BiLSTMModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, target_padding_value=pos_padding_value):  #, embedder):
        super(BiLSTMModel, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        # self.embedder = embedder
        # self.embedding_layer = nn.Embedding.from_pretrained(embedder.vectors)

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to the bidirectionality

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc(lstm_out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()


In [703]:
output_dim = len(df_train["POS"].unique()) + 1  # +1 for padding
input_dim = EMBEDDING_DIM

In [704]:
baseline_version = 1

In [705]:
hidden_dim = 28
max_epochs = 5

save_dir = "logs"
load_model = False
train_model = True
model_ckpt = "baseline.ckpt"

PATH = os.path.join(save_dir, "lightning_logs", "version_" + str(baseline_version), "checkpoints", model_ckpt)

if load_model:
    model = BiLSTMModel.load_from_checkpoint(PATH, input_dim=input_dim, hidden_dim=hidden_dim,
                                             output_dim=output_dim)  #, embedder=embedder)
else:
    model = BiLSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)  #, embedder=embedder)

if train_model:
    tb_logger = TensorBoardLogger(version=baseline_version, save_dir='logs')
    trainer = pl.Trainer(max_epochs=max_epochs, logger=tb_logger, default_root_dir='./', log_every_n_steps=1)
    trainer.fit(model, train_loader, val_loader)
    trainer.save_checkpoint(PATH)
    baseline_version += 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory logs\lightning_logs\version_1\checkpoints exists and is not empty.

  | Name             | Type          | Params
---------------------------------------------------
0 | lstm             | LSTM          | 17.9 K
1 | fc               | Linear        | 2.6 K 
2 | _train_f1_metric | F1ScoreCustom | 0     
3 | _val_f1_metric   | F1ScoreCustom | 0     
4 | _test_f1_metric  | F1ScoreCustom | 0     
---------------------------------------------------
20.5 K    Trainable params
0         Non-trainable params
20.5 K    Total params
0.082     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [706]:
# test the model
trainer.test(model, dataloaders=test_loader)

C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          step                      5.0
         test_f1            0.5615873336791992
        test_loss           1.4257254600524902
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.4257254600524902, 'step': 5.0, 'test_f1': 0.5615873336791992}]

In [707]:
%reload_ext tensorboard

In [708]:
%tensorboard --logdir logs

Launching TensorBoard...

### Model 1: Bidirectional 2-layers LSTM + Dense layer

In [709]:
class Model1(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, target_padding_value=pos_padding_value):  #, embedder):
        super(Model1, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        # self.embedder = embedder
        # self.embedding_layer = nn.Embedding.from_pretrained(embedder.vectors)

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to the bidirectionality

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc(lstm_out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()

In [710]:
model1_version = 1

In [723]:
hidden_dim = 28
max_epochs = 10

save_dir = "logs"
load_model = False
train_model = True
model_ckpt = "model1.ckpt"

PATH = os.path.join(save_dir, "lightning_logs", "version_" + str(model1_version), "checkpoints", model_ckpt)

if load_model:
    model_1 = Model1.load_from_checkpoint(PATH, input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
else:
    model_1 = Model1(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

if train_model:
    tb_logger = TensorBoardLogger(version=model1_version, save_dir='logs')
    trainer = pl.Trainer(max_epochs=max_epochs, logger=tb_logger, default_root_dir='./', log_every_n_steps=1)
    trainer.fit(model_1, train_loader, val_loader)
    trainer.save_checkpoint(PATH)
    model1_version += 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory logs\lightning_logs\version_2\checkpoints exists and is not empty.

  | Name             | Type          | Params
---------------------------------------------------
0 | lstm             | LSTM          | 37.2 K
1 | fc               | Linear        | 2.6 K 
2 | _train_f1_metric | F1ScoreCustom | 0     
3 | _val_f1_metric   | F1ScoreCustom | 0     
4 | _test_f1_metric  | F1ScoreCustom | 0     
---------------------------------------------------
39.8 K    Trainable params
0         Non-trainable params
39.8 K    Total params
0.159     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [728]:
trainer.test(model_1, dataloaders=test_loader);

C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          step                     10.0
         test_f1            0.6905661225318909
        test_loss           0.8774994015693665
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


### Model 2: Bidirectional LSTM + 2 Dense layers

In [729]:
class Model2(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, fc_size,
                 target_padding_value=pos_padding_value):  #, embedder):
        super(Model2, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        # self.embedder = embedder
        # self.embedding_layer = nn.Embedding.from_pretrained(embedder.vectors)

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.fc_1 = nn.Linear(hidden_dim * 2, fc_size)  # Multiplied by 2 due to bidirectionality
        self.fc_2 = nn.Linear(fc_size, output_dim)

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc_1(lstm_out)
        out = self.fc_2(out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()

In [730]:
model2_version = 1

In [731]:
hidden_dim = 28
fc_size = 100

max_epochs = 10

save_dir = "logs"
load_model = False
train_model = True
model_ckpt = "model2.ckpt"

PATH = os.path.join(save_dir, "lightning_logs", "version_" + str(model2_version), "checkpoints", model_ckpt)

if load_model:
    model_2 = Model2.load_from_checkpoint(PATH, input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim,
                                          fc_size=fc_size)
else:
    model_2 = Model2(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, fc_size=fc_size)

if train_model:
    tb_logger = TensorBoardLogger(version=model2_version, save_dir='logs')
    trainer = pl.Trainer(max_epochs=max_epochs, logger=tb_logger, default_root_dir='./', log_every_n_steps=1)
    trainer.fit(model_2, train_loader, val_loader)
    trainer.save_checkpoint(PATH)
    model2_version += 1

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory logs\lightning_logs\version_1\checkpoints exists and is not empty.

  | Name             | Type          | Params
---------------------------------------------------
0 | lstm             | LSTM          | 37.2 K
1 | fc_1             | Linear        | 5.7 K 
2 | fc_2             | Linear        | 4.6 K 
3 | _train_f1_metric | F1ScoreCustom | 0     
4 | _val_f1_metric   | F1ScoreCustom | 0     
5 | _test_f1_metric  | F1ScoreCustom | 0     
---------------------------------------------------
47.5 K    Trainable params
0         Non-trainable params
47.5 K    Total params
0.190     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\Matteo\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [716]:
trainer.test(model_2, dataloaders=test_loader);

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          step                      0.0
         test_f1            0.06370722502470016
        test_loss           3.8354685306549072
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 3.8354685306549072,
  'step': 0.0,
  'test_f1': 0.06370722502470016}]

In [717]:
%reload_ext tensorboard
%tensorboard --logdir logs

Launching TensorBoard...

## TASK 4: Metrics

* Assign static embedding to validation and test set OOV words (Edo: see unk_init param at [docs](https://torchtext.readthedocs.io/en/latest/vocab.html))
* **Concatenate** all tokens in a data split -> (**Hint**: accumulate FP, TP, FN, TN iteratively) TODO: rewrite 
* Evaluate models using macro F1-score, computed over **all** tokens.
* TODO: remove -> **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

In [718]:
word_iterator = (word for word in corpus_df['text'].unique())
word_vocab = build_vocab_from_iterator(word_iterator)

In [719]:
# OOV words
existing_vocab_words = set(word_vocab.get_itos())

val_oov_words = set(df_val['text']) - existing_vocab_words  # OOV words in validation set
test_oov_words = set(df_test['text']) - existing_vocab_words  # OOV words in test set

print("Out-of-vocabulary words in validation set:", val_oov_words)
print('len:', len(val_oov_words))
print("Out-of-vocabulary words in test set:", test_oov_words)
print('len:', len(test_oov_words))

Out-of-vocabulary words in validation set: {'authorities', 'vehicles', 'advertised', 'biggest', 'wizards', 'interests', 'provided', 'Research', '396,000', 'suggest', 'conceivable', 'coal', 'Seoul', 'assault', 'event', 'Due', 'aesthetic', 'heebie-jeebies', 'technically', 'Hoylake', 'pre-existing', 'speculative', 'How', 'companion', '62%-owned', 'restricting', 'intense', 'Pretty', 'century', 'House-Senate', 'disappointment', 'Express-Buick', 'dealers', '107.03', 'tree', 'syndicate', 'economist', 'cancer', 'Ballot', 'skeptical', 'directly', 'Secretary', 'soybeans', 'outnumbered', 'although', 'consequence', 'abridging', 'thus', 'Tots', 'attributed', 'Lawless', 'pain', '1.6', 'estimate', 'blurred', 'tied', 'Public', 'lines', "O'Loughlin", 'curses', 'terminated', 'ease', 'headed', 'surpluses', '29', 'ranged', 'purchase', 'complex', 'every', 'reasoning', 'anticipation', 'Belt', 'Something', 'mammoth', '3.43', 'shudders', 'mirror', 'hotel', 'income', 'propose', 'meals', 'records', 'drink', 'to

In [720]:
# Static embeddings
static_embeddings_val = {}
static_embeddings_test = {}

# Assign static embeddings to OOV words in the validation set
for word in val_oov_words:
    if word in embedder.stoi:  #For each word assign a static embedding
        static_embeddings_val[word] = embedder[word]

# Assign static embeddings to OOV words in the test set
for word in test_oov_words:
    if word in embedder.stoi:
        static_embeddings_test[word] = embedder[word]


In [721]:
num_oov_words_to_display = 3

# print to check the validation set oov embeddings
print(f"Static embeddings for OOV words in the validation set:")
for i, (word, embedding) in enumerate(static_embeddings_val.items()):
    if i < num_oov_words_to_display:
        print(f"Word: {word}, Embedding: {embedding}")
    else:
        break



Static embeddings for OOV words in the validation set:
Word: authorities, Embedding: tensor([ 0.8391, -0.9735,  0.0234, -0.1850,  0.0184, -0.5754, -0.4064,  0.8554,
         0.1494, -0.5117,  0.6171, -0.4182,  0.1502, -0.2079,  0.6957,  0.0634,
        -0.3153, -0.5238,  0.1286,  0.0235,  0.3934,  0.5191,  0.1238,  0.1504,
        -0.4713, -2.5485,  0.2704, -0.1089, -0.1735, -0.2373,  2.9339, -0.1805,
        -0.9292, -1.1758,  0.0700,  0.3043,  0.2473, -0.5854, -0.0745,  0.6907,
        -0.4885,  0.5325,  1.1832, -0.0712,  0.4329, -0.7524, -1.0586,  0.8434,
         0.6690, -1.1813])
Word: vehicles, Embedding: tensor([ 0.7598, -0.7656,  2.0944, -0.3748, -0.3495,  0.1849, -1.1152, -1.0155,
         0.2449, -0.7160,  0.6036, -1.0472, -0.2830, -0.3622,  0.2996,  0.0435,
        -0.3185,  1.4753, -0.4976, -2.1802,  0.5287, -0.3492, -0.7874, -0.0588,
        -0.1199, -0.5924, -0.1937,  0.4254,  1.2132,  0.1945,  2.6633,  0.3081,
        -0.1981, -0.2880,  1.1756,  0.6820,  0.4655, -0.3504,

In [722]:
# print to check the test set oov embeddings
print(f"Static embeddings for OOV words in the test set:")
for i, (word, embedding) in enumerate(static_embeddings_test.items()):
    if i < num_oov_words_to_display:
        print(f"Word: {word}, Embedding: {embedding}")
    else:
        break

Static embeddings for OOV words in the test set:
Word: newspapers, Embedding: tensor([-0.5160,  0.1002,  0.3813,  0.0117, -0.4833, -0.6482, -1.5710, -0.4006,
        -0.2882, -0.3237, -0.7641, -0.3429,  0.2339,  0.0084,  0.5827, -0.5355,
        -1.0689, -0.3693,  0.2158,  0.0684,  1.1848,  0.5830,  0.8454,  0.8553,
        -0.0994, -1.5168, -1.0059,  0.1249, -0.3572,  0.1986,  2.4222, -0.1785,
         0.5885, -0.3776, -1.2228, -0.2266, -0.3435, -0.4328, -0.7428,  0.2716,
         1.2048,  0.5102,  0.1840,  0.4308,  0.0978,  0.4768, -1.5664,  0.9066,
         0.1876, -0.5749])
Word: sharply, Embedding: tensor([-3.4411e-01, -1.1243e+00,  5.6112e-01, -8.5382e-01, -8.2236e-02,
         2.3612e-01, -1.7791e-01, -8.0127e-01, -2.5020e-01,  8.3882e-02,
        -6.4719e-02, -7.8313e-01, -3.1500e-01, -2.9628e-01, -5.8452e-02,
         6.0357e-01, -1.0127e+00, -1.3719e+00, -8.4500e-04, -6.6783e-01,
         7.6387e-01, -2.0251e-01,  3.5552e-01, -8.9121e-01,  6.7238e-02,
        -1.3530e+00,  6.

## TASK 5: Training and Evaluation

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

## TASK 6: Error analysis

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible) 
* Comment about errors and propose possible solutions on how to address them.

# TODO: REPORT